In [2]:
!pip install lifelines

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.2/349.2 kB 8.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 6.4 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=608ec4a96f733e1e6f37391bb4b7f350cb4fa206aef50f1226701658d44b61d0
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma


In [42]:
import pandas as pd
import torch
import numpy as np
import math
import os

from torch import nn
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from lifelines.utils import concordance_index

In [4]:
!wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/DataFusion2024/Data/clients.csv
!wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/DataFusion2024/Data/train.csv
!wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/DataFusion2024/Data/report_dates.csv
!wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/DataFusion2024/Data/transactions.csv.zip

--2024-04-25 09:18:08--  https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/DataFusion2024/Data/clients.csv
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2457605 (2.3M) [text/csv]
Saving to: 'clients.csv'

clients.csv         100%[===================>]   2.34M  1.85MB/s    in 1.3s    

2024-04-25 09:18:10 (1.85 MB/s) - 'clients.csv' saved [2457605/2457605]

--2024-04-25 09:18:11--  https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/DataFusion2024/Data/train.csv
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 754462 (737K) [text/csv]
Saving to: 'train.csv'

t

In [5]:
!unzip transactions.csv

Archive:  transactions.csv.zip
  inflating: transactions.csv        


# Data Loading

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"
device

device(type='cuda')

In [7]:
clients = pd.read_csv("clients.csv")
report_dates = pd.read_csv("report_dates.csv", parse_dates=["report_dt"])
train = pd.read_csv("train.csv")
transactions = pd.read_csv("transactions.csv", parse_dates=["transaction_dttm"])

# EDA and PreProcessing

In [8]:
clients.fillna({"employee_count_nm": 7}, inplace=True)
train.merge(clients, on="user_id").groupby("employee_count_nm").agg({"target":lambda x: [x.value_counts()[1]/len(x) * 100, x.value_counts()[1], x.value_counts()[0]]}).sort_values("target")

,target
employee_count_nm,
БОЛЕЕ 500,"[3.7065052950075645, 49, 1273]"
БОЛЕЕ 1001,"[4.1140837960625944, 489, 11397]"
ОТ 501 ДО 1000,"[4.667739710278225, 203, 4146]"
ОТ 101 ДО 500,"[5.491057420771886, 525, 9036]"
ОТ 51 ДО 100,"[5.653853976001627, 278, 4639]"
ОТ 11 ДО 50,"[6.507666098807496, 191, 2744]"
ОТ 11 ДО 30,"[7.026143790849673, 86, 1138]"
ДО 10,"[7.159264931087289, 187, 2425]"
ОТ 31 ДО 50,"[8.135942327497425, 79, 892]"


In [9]:
clients.replace({'employee_count_nm':{'ОТ 101 ДО 500':4,'БОЛЕЕ 1001':6,'ОТ 501 ДО 1000':5,'ДО 10':0,
                                      'ОТ 11 ДО 50':2,'ОТ 51 ДО 100':3,'БОЛЕЕ 500':5,'ОТ 11 ДО 30':2,
                                      'ОТ 31 ДО 50':2}}, inplace=True)
clients.head()

/tmp/ipykernel_35/2213134391.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  clients.replace({'employee_count_nm':{'ОТ 101 ДО 500':4,'БОЛЕЕ 1001':6,'ОТ 501 ДО 1000':5,'ДО 10':0,


,user_id,report,employee_count_nm,bankemplstatus,customer_age
0,3,2,4,0,3
1,9,1,6,0,3
2,13,6,5,0,2
3,37,5,6,0,2
4,41,1,4,0,2


In [10]:
train.merge(clients, on="user_id").groupby("bankemplstatus").agg({"target":lambda x: x.value_counts()[1]/len(x) * 100})

,target
bankemplstatus,
0,8.489827
1,0.485437


Работники банка редко уходят в отток.\
Bank employees rarely churn.

In [11]:
transactions.head(5)

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm
0,3,3,1,-183.883957,2022-01-28 12:05:33
1,3,3,1,-3206.437012,2022-01-28 12:52:30
2,3,16,1,-153866.890625,2022-02-16 14:45:56
3,3,56,1,-15144.601562,2022-03-09 19:58:29
4,3,0,1,5297.908691,2022-03-12 18:11:31


In [12]:
data = clients.merge(train, on="user_id", how="left")
data = data.merge(report_dates, on="report")

In [13]:
transactions['date'] = transactions['transaction_dttm'].dt.date.astype('datetime64[ns]')

In [14]:
transactions_upd = transactions.merge(data[["report_dt", "user_id"]], on="user_id")
transactions_upd["diff_days"] = (transactions_upd['report_dt']-transactions_upd['transaction_dttm']) / np.timedelta64(1, 'D')
some_data = transactions_upd.groupby("user_id").agg(trans_sum=("transaction_amt", "sum"),
                                                    trans_std=("transaction_amt", "std"),
                                                    trans_min=("transaction_amt", "min"),
                                                    trans_max=("transaction_amt", "max"),
                                                    trans_mean=("transaction_amt", "mean"),
                                                    trans_count=("transaction_amt", "count"),
                                                    trans_pos_sum=("transaction_amt", lambda x: x[(x>0)].sum()),
                                                    trans_neg_sum=("transaction_amt", lambda x: x[(x<0)].sum()),
                                                    trans_pos_mean=("transaction_amt", lambda x: x[(x > 0)].mean() if (x > 0).sum() > 0 else 0),
                                                    trans_neg_mean=("transaction_amt", lambda x: x[(x < 0)].mean() if (x < 0).sum() > 0 else 0),
                                                    trans_first=("transaction_amt", lambda x: x.iloc[-1]),
                                                    trans_last=("transaction_amt", lambda x: x.iloc[0]),
                                                    diff_days_max=("diff_days", "max"),
                                                    diff_days_min=("diff_days", "min"),
                                                    mcc_code_nunique=("mcc_code", "nunique"),
                                                    mcc_code_count=("mcc_code", "count"),
                                                    currency_rk_nunique=("currency_rk", "nunique"),
                                                    currency_rk_count=("currency_rk", "count"),
                                                    currency_rk_median=("currency_rk", "median"),
                                                    currency_rk_min=("currency_rk", "min"),
                                                    currency_rk_max=("currency_rk", "max"),
                                                    date_max=("date", lambda x: x.max().day),
                                                    date_min=("date", lambda x: x.min().day),
                                                    date=("date", lambda x: (x.max()-x.min()).days))


In [15]:
date_trans = transactions.groupby("user_id").agg({"date":"unique"}).explode("date").reset_index()
date_trans["interval"] = date_trans.groupby("user_id")["date"].diff()
date_trans = date_trans.dropna()
date_trans["interval"] = date_trans["interval"].dt.days
date_trans = date_trans.groupby("user_id").agg({"interval":["last","max"]})
date_trans.columns = list( map(lambda x: "date_"+str(x), date_trans.columns ) )

In [16]:
data = data.merge(some_data, on="user_id")
data = data.merge(date_trans, on="user_id")

In [17]:
data = (
                data
                .assign(hour=data.report_dt.dt.hour)
                .assign(day=data.report_dt.dt.day)
                .assign(month=data.report_dt.dt.month)
                .assign(day_of_week=data.report_dt.dt.dayofweek)
                .assign(week_of_year=data.report_dt.dt.isocalendar().week)
              )

def generate_cyclical_features(data, col_name, period, start_num=0):
    kwargs = {
        f'sin_{col_name}' : lambda x: np.sin(2*np.pi*(data[col_name]-start_num)/period),
        f'cos_{col_name}' : lambda x: np.cos(2*np.pi*(data[col_name]-start_num)/period)
             }
    return data.assign(**kwargs).drop(columns=[col_name])

data = generate_cyclical_features(data, 'hour', 24, 0)
data = generate_cyclical_features(data, 'day_of_week', 7, 0)
data = generate_cyclical_features(data, 'month', 12, 1)
data = generate_cyclical_features(data, 'week_of_year', 52, 0)

data.drop(columns=["report_dt"], inplace=True)

In [18]:
def preprocess(x):
    return (np.log(x.abs().sum()))

temp_tran = transactions.groupby(['user_id','currency_rk'])['transaction_amt'].agg([preprocess])
temp_tran = temp_tran.unstack('currency_rk').fillna(0)
temp_tran.columns = ['logsumabs_0','logsumabs_1','logsumabs_2','logsumabs_3']

In [19]:
data = data.merge(temp_tran, on="user_id")

In [20]:
data.head(3)

,user_id,report,employee_count_nm,bankemplstatus,customer_age,target,time,trans_sum,trans_std,trans_min,...,sin_day_of_week,cos_day_of_week,sin_month,cos_month,sin_week_of_year,cos_week_of_year,logsumabs_0,logsumabs_1,logsumabs_2,logsumabs_3
0,3,2,4,0,3,0.0,77.0,13706.416641,60860.880477,-153866.890625,...,0.974928,-0.222521,-5.000000e-01,-0.866025,-0.885456,-0.464723,0.00000,12.789712,0.0,0.0
1,9,1,6,0,3,NaN,NaN,-323434.666813,11797.055119,-90147.617188,...,-0.781831,0.623490,1.224647e-16,-1.000000,-0.464723,-0.885456,0.00000,12.686752,0.0,0.0
2,13,6,5,0,2,0.0,86.0,-124717.379150,23507.953991,-58740.300781,...,-0.974928,-0.222521,-5.000000e-01,0.866025,0.0,1.0,9.28478,12.825245,0.0,0.0


In [21]:
clients

,user_id,report,employee_count_nm,bankemplstatus,customer_age
0,3,2,4,0,3
1,9,1,6,0,3
2,13,6,5,0,2
3,37,5,6,0,2
4,41,1,4,0,2
...,...,...,...,...,...
95995,562043,12,7,0,2
95996,562205,12,7,0,1
95997,562312,12,7,0,0
95998,562721,12,7,0,2


In [22]:
transactions

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm,date
0,3,3,1,-183.883957,2022-01-28 12:05:33,2022-01-28
1,3,3,1,-3206.437012,2022-01-28 12:52:30,2022-01-28
2,3,16,1,-153866.890625,2022-02-16 14:45:56,2022-02-16
3,3,56,1,-15144.601562,2022-03-09 19:58:29,2022-03-09
4,3,0,1,5297.908691,2022-03-12 18:11:31,2022-03-12
...,...,...,...,...,...,...
13075018,562740,155,1,-2484.366211,2023-03-20 11:52:09,2023-03-20
13075019,562740,9,1,-187.658463,2023-03-20 12:10:22,2023-03-20
13075020,562740,1,1,-891.933350,2023-03-20 15:53:37,2023-03-20
13075021,562740,13,1,-464.467316,2023-03-20 15:54:49,2023-03-20


In [23]:
transactions_upd["sm"] = np.where(transactions_upd['transaction_amt']<0, transactions_upd['transaction_amt'],0)
transactions_upd['sp'] = np.where( transactions_upd['transaction_amt']>0, transactions_upd['transaction_amt'],0)
transactions_upd['m'] = (transactions_upd['report_dt'].dt.year-
                     transactions_upd['transaction_dttm'].dt.year)*12+(transactions_upd['report_dt'].dt.month-
                                                                   transactions_upd['transaction_dttm'].dt.month)

msump = transactions_upd.pivot_table(values="sp",
    index="user_id",
    columns="m",
    aggfunc='sum',
    fill_value=0,)
msump.columns = list( map(lambda x: "msump"+str(x), msump.columns ) )
msump

,msump3,msump4,msump5,msump6,msump7,msump8,msump9
user_id,,,,,,,
3,137734.573242,43075.747864,5297.908691,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
13,10805.421875,10952.112305,0.000000,86198.161377,10038.188965,10772.799805,0.000000
37,0.000000,0.000000,5487.140625,5251.647949,0.000000,0.000000,0.000000
41,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
562043,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
562205,319.916626,0.000000,0.000000,0.000000,0.000000,0.000000,1648.989708
562312,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [24]:
msumm = transactions_upd.pivot_table(values="sm",
    index="user_id",
    columns="m",
    aggfunc='sum',
    fill_value=0,)
msumm.columns = list( map(lambda x: "msumm"+str(x), msumm.columns ) )
msumm

,msumm3,msumm4,msumm5,msumm6,msumm7,msumm8,msumm9
user_id,,,,,,,
3,0.000000,0.000000,-15144.601562,-153866.890625,-3390.320969,0.000000,0.000000
9,-13289.485764,-104223.187939,-109800.056656,-54146.384140,-18500.321304,-12387.577271,-11087.653740
13,-16394.193359,-27095.248047,-27650.769531,-76186.732422,-33908.388672,-53024.051758,-19224.679688
37,-32719.820328,-41364.700729,-67081.604048,-53322.076138,-69769.034760,-50056.823341,-28284.328693
41,-8045.445801,-17708.008820,-30354.633301,-17462.199585,-18175.117676,-16841.208984,0.000000
...,...,...,...,...,...,...,...
562043,0.000000,-142.056610,0.000000,-2984.231926,-11080.026783,-15374.940796,0.000000
562205,-10452.779190,-7546.495855,-6601.707232,-4821.073084,-6082.155629,-5607.360139,-1349.243835
562312,0.000000,-2644.634823,-1362.274536,-3165.544842,-3938.301605,-4495.630554,-2931.434910


In [25]:
data.head()

,user_id,report,employee_count_nm,bankemplstatus,customer_age,target,time,trans_sum,trans_std,trans_min,...,sin_day_of_week,cos_day_of_week,sin_month,cos_month,sin_week_of_year,cos_week_of_year,logsumabs_0,logsumabs_1,logsumabs_2,logsumabs_3
0,3,2,4,0,3,0.0,77.0,13706.416641,60860.880477,-153866.890625,...,0.974928,-0.222521,-5.000000e-01,-0.866025,-0.885456,-0.464723,0.00000,12.789712,0.0,0.0
1,9,1,6,0,3,NaN,NaN,-323434.666813,11797.055119,-90147.617188,...,-0.781831,0.623490,1.224647e-16,-1.000000,-0.464723,-0.885456,0.00000,12.686752,0.0,0.0
2,13,6,5,0,2,0.0,86.0,-124717.379150,23507.953991,-58740.300781,...,-0.974928,-0.222521,-5.000000e-01,0.866025,0.0,1.0,9.28478,12.825245,0.0,0.0
3,37,5,6,0,2,0.0,89.0,-331859.599463,3913.114835,-35782.984375,...,0.974928,-0.222521,-8.660254e-01,0.500000,-0.464723,0.885456,0.00000,12.775178,0.0,0.0
4,41,1,4,0,2,0.0,57.0,-108586.614166,5251.737982,-16841.208984,...,-0.781831,0.623490,1.224647e-16,-1.000000,-0.464723,-0.885456,0.00000,11.595303,0.0,0.0


In [26]:
data = data.join(msumm, on="user_id", how="left")

In [27]:
train_data = data[~data["target"].isna()]
train_data.reset_index(drop=True, inplace=True)

In [28]:
test_data = data[data["target"].isna()]
test_data.reset_index(drop=True, inplace=True)

In [29]:
train_user, val_user = train_test_split(train_data.index, test_size=0.2,
                                        stratify=train_data.target)
val_user.shape, train_user.shape

((12800,), (51199,))

In [30]:
train_data.iloc[train_user].target.value_counts(), train_data.iloc[val_user].target.value_counts()

(target
 0.0    46865
 1.0     4334
 Name: count, dtype: int64,
 target
 0.0    11717
 1.0     1083
 Name: count, dtype: int64)

In [31]:
transactions.head(3)

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm,date
0,3,3,1,-183.883957,2022-01-28 12:05:33,2022-01-28
1,3,3,1,-3206.437012,2022-01-28 12:52:30,2022-01-28
2,3,16,1,-153866.890625,2022-02-16 14:45:56,2022-02-16


In [32]:
transactions.set_index("transaction_dttm", inplace=True)
transactions.drop(columns="date", inplace=True)
group_transactions = transactions.groupby("user_id")

In [33]:
group_transactions.get_group(3)

,user_id,mcc_code,currency_rk,transaction_amt
transaction_dttm,,,,
2022-01-28 12:05:33,3,3,1,-183.883957
2022-01-28 12:52:30,3,3,1,-3206.437012
2022-02-16 14:45:56,3,16,1,-153866.890625
2022-03-09 19:58:29,3,56,1,-15144.601562
2022-03-12 18:11:31,3,0,1,5297.908691
2022-04-11 20:48:26,3,0,1,534.363098
2022-04-11 20:53:35,3,0,1,4549.455078
2022-04-29 18:42:58,3,0,1,37991.929688
2022-05-05 20:32:38,3,0,1,28335.613281


# Pytorch

## EarlyStoping

Class for stopping the model early

In [43]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.result = {}
        self.trace_func = trace_func

    def __call__(self, val_loss, ci, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, ci, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, ci, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, ci, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        self.result[self.path] = ci
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss


## DataLoader

Class for data conversion

In [249]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, data,
                 transactions,
                 test=False):
        self.data = data
        self.transactions = transactions
        self.test = test

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        user_info = self.data.iloc[idx]

        # RNN input
        rnn_tensor = torch.from_numpy(self.transactions.get_group(user_info.user_id).drop(["user_id"], axis=1).to_numpy().astype(np.float32))
        rnn_tensor = torch.cat([rnn_tensor, torch.zeros(size=(1497-rnn_tensor.shape[0], rnn_tensor.shape[1]))]).to(torch.float32)

        # Linear input
        if not self.test:
            linear_input = torch.tensor(user_info.drop(["target", "time", "user_id"], axis=0).values).to(torch.float32)
        else:
            linear_input = torch.tensor(user_info.drop(["user_id"], axis=0).values).to(torch.float32)
            

        if self.test:
            return (rnn_tensor, linear_input)
        else:
            return (rnn_tensor, linear_input),\
                    torch.tensor([torch.tensor(user_info.time), torch.tensor(user_info.target)])
        

In [160]:
!mkdir ./models

mkdir: cannot create directory './models': File exists


## Model Training

### RNN

Implementation of the model

In [205]:
class RNN_big(nn.Module):
    def __init__(self,
                 input_rnn,
                 input_linear,
                 hidden_dim,
                 in1,
                 in2,
                 output_dim,
                 rnn_layers=2):
        super().__init__()
        self.batch_size = batch_size
        self.hidden_dim = hidden_dim
        self.rnn_layers = rnn_layers

        self.norm_rnn = nn.BatchNorm1d(1497)
        self.rnn1 = nn.GRU(input_rnn, hidden_dim, rnn_layers,
                            batch_first=True, bidirectional=True)

        self.fc1 = nn.Linear(hidden_dim*2, in2)
        self.fc2 = nn.Linear(hidden_dim*2, in2)

        self.norm1 = nn.BatchNorm1d(input_linear)
        self.linear1 = nn.Linear(input_linear, in1)
        self.linear2 = nn.Linear(in1, in2)

        self.out = nn.Linear(in2, output_dim)

    def forward(self, x1, x2, h):
        x1 = self.norm_rnn(x1)
        out_rnn, h = self.rnn1(x1, h)
        out_rnn = F.relu(out_rnn[:, -1])

        linear_1 = F.relu(self.fc1(out_rnn))
        linear_2 = F.relu(self.fc2(out_rnn))

        linear_features = F.relu(self.linear1(self.norm1(x2)))
        linear_features = F.relu(self.linear2(linear_features))
        hidden = linear_1 + linear_2 + linear_features

        out = self.out(hidden)
        return F.sigmoid(out), h

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.rnn_layers*2, batch_size, self.hidden_dim).zero_().to(device)
        return hidden


### Train

In [221]:
batch_size = 64
val_dataset = CustomDataset(train_data.iloc[val_user], group_transactions)
val_loader = DataLoader(val_dataset, batch_size=batch_size, drop_last=True)

In [222]:
def train_model(model, loader, optimizer, criterion, epochs, path="./", patience=10):
    result = {}
    early_stop = EarlyStopping(patience=min(patience, epochs), path=path)

    for epoch in range(epochs):
        model.train()
        print("EPOCH ", epoch+1)

        epoch_loss = 0
        y_true, y_preds, times = [], [], []
        h = model.init_hidden(batch_size)

        for x, y in tqdm(loader):
            h = h.data
            x1, x2 = x
            time, y_batch = y[:, 0], y[:, 1]
            x1, x2, y_batch = x1.to(device), x2.to(device), y_batch.to(device)

            optimizer.zero_grad()
            predictions, h = model(x1, x2, h)

            loss = criterion(predictions, torch.unsqueeze(y_batch, -1).float())
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

            times.extend(time)
            y_true.extend(y_batch.detach().cpu())
            y_preds.extend(predictions.detach().cpu())

        print("Loss:", epoch_loss / len(loader))
        print("CI:", concordance_index(np.array(times), -np.array(y_preds), np.array(y_true)))
        print("ROC_AUC:", roc_auc_score(y_true, y_preds), "\n")

        val_loss, ci = test(model, val_loader, criterion)
        early_stop(val_loss, ci, model)
        result.update(early_stop.result)

        if early_stop.early_stop:
              break

    return result


def test(model, loader, criterion=nn.BCELoss()):
    model.eval()
    loss, CI = 0, 0
    y_true, y_preds, times = [], [], []
    h = model.init_hidden(batch_size)

    with torch.no_grad():
        for x, y in tqdm(loader):
            x1, x2 = x
            time, y_batch = y[:, 0], y[:, 1]
            x1, x2, y_batch = x1.to(device), x2.to(device), y_batch.to(device)

            predictions, h = model(x1, x2, h)
            loss += criterion(predictions, torch.unsqueeze(y_batch, -1).float()).item()

            times.extend(time)
            y_true.extend(y_batch.detach().cpu())
            y_preds.extend(predictions.detach().cpu())

        val_loss = loss/len(loader)
        ci = concordance_index(np.array(times), -np.array(y_preds), np.array(y_true))
        print("Loss:", val_loss)
        print("CI:", ci)
        print("ROC_AUC:", roc_auc_score(y_true, y_preds), "\n")

    return val_loss, ci


In [223]:
train_dataset = CustomDataset(train_data.iloc[train_user], group_transactions)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [224]:
got_data = next(iter(train_loader))[0]
rnn_dim = got_data[0].shape[2]
linear_dim = got_data[1].shape[1]
hidden_dim = 16
in1 = 512
in2 = 128

model = RNN_big(rnn_dim, linear_dim, hidden_dim, in1, in2, 1, 4).to(device)
sum(p.numel() for p in model.parameters())

119863

In [225]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.BCELoss()

In [226]:
train_model(model, train_loader, optimizer, criterion, 100, path="./models/model.pt")

EPOCH  1


100%|██████████| 799/799 [02:32<00:00,  5.24it/s]


Loss: 0.27351657111892413
CI: 0.6888856304437417
ROC_AUC: 0.6834030708015219 



100%|██████████| 200/200 [00:24<00:00,  8.01it/s]


Loss: 0.2651889868825674
CI: 0.7245874251901537
ROC_AUC: 0.715351836646818 

EPOCH  2


...

100%|██████████| 799/799 [02:30<00:00,  5.31it/s]


Loss: 0.2385160260005051
CI: 0.7904274752901356
ROC_AUC: 0.7928392880447331 



100%|██████████| 200/200 [00:25<00:00,  7.85it/s]


Loss: 0.25863066654652356
CI: 0.7416942805801623
ROC_AUC: 0.7373929539128813 

EarlyStopping counter: 10 out of 10


{'./models/model.pt': 0.7481186954392047}

###Submmit

In [250]:
y_pred = []
test_dataset = CustomDataset(test_data.sort_values("user_id"), group_transactions, True)
test_loader = DataLoader(test_dataset, batch_size=128)

In [256]:
model.eval()
model.to(device)
h = model.init_hidden(128)
for x in tqdm(test_loader):
    x1, x2 = x
    predictions, _ = model(x1.to(device), x2.to(device), h)
    predictions = predictions.squeeze(0).to(torch.float).detach().cpu().numpy()
    y_pred.extend(predictions)


100%|██████████| 250/250 [00:55<00:00,  4.47it/s]


In [257]:
!wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/DataFusion2024/Data/sample_submit_naive.csv
submission = pd.read_csv("sample_submit_naive.csv")

--2024-04-25 12:14:15--  https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/DataFusion2024/Data/sample_submit_naive.csv
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 788735 (770K) [text/csv]
Saving to: 'sample_submit_naive.csv'

sample_submit_naive 100%[===================>] 770.25K   809KB/s    in 1.0s    

2024-04-25 12:14:17 (809 KB/s) - 'sample_submit_naive.csv' saved [788735/788735]



In [258]:
preds = []
for i in y_pred:
    preds.append(i[0])

In [259]:
submission.predict = preds

In [260]:
submission.head()

,user_id,predict
0,9,0.032000
1,61,0.006418
2,62,0.055881
3,80,0.008052
4,88,0.140380


In [ ]:
submission.to_csv("submission.csv", index=False)